<a href="https://colab.research.google.com/github/sanaaria/Hyperparameter-Optimization/blob/main/hyperparameters_optimization_bbbp_lastEdition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install dragonfly-opt
# !pip install -U hyperopt
# !pip install hpbandster ConfigSpace
# !pip install optuna

In [2]:
#pip install bayesian-optimization

In [1]:
#!pip install optuna

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
from pandas import read_csv
from sklearn.impute import SimpleImputer

In [3]:
from google.colab import files
uploaded = files.upload()

Saving bbbp.csv to bbbp (1).csv


In [165]:
import pandas as pd
import io
data = pd.read_csv(io.BytesIO(uploaded['bbbp.csv']))
data.head()

,smiles,"('RDKit2DNormalized_calculated', <class 'bool'>)","('BalabanJ', <class 'numpy.float64'>)","('BertzCT', <class 'numpy.float64'>)","('Chi0', <class 'numpy.float64'>)","('Chi0n', <class 'numpy.float64'>)","('Chi0v', <class 'numpy.float64'>)","('Chi1', <class 'numpy.float64'>)","('Chi1n', <class 'numpy.float64'>)","('Chi1v', <class 'numpy.float64'>)",...,"('fr_sulfone', <class 'numpy.float64'>)","('fr_term_acetylene', <class 'numpy.float64'>)","('fr_tetrazole', <class 'numpy.float64'>)","('fr_thiazole', <class 'numpy.float64'>)","('fr_thiocyan', <class 'numpy.float64'>)","('fr_thiophene', <class 'numpy.float64'>)","('fr_unbrch_alkane', <class 'numpy.float64'>)","('fr_urea', <class 'numpy.float64'>)","('qed', <class 'numpy.float64'>)",p_np
0,[Cl].CC(C)NCC(O)COc1cccc2ccccc12,True,0.000020,0.111833,0.106451,0.177350,0.187815,0.110054,0.168672,0.119391,...,5.770000e-14,2.960000e-11,0.168378,0.16738,1.480000e-18,2.320000e-16,4.700000e-08,0.166633,0.930688,1
1,C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl,True,0.921082,0.077697,0.324169,0.379878,0.467164,0.240353,0.344156,0.422452,...,5.770000e-14,2.960000e-11,0.168378,0.16738,1.480000e-18,2.320000e-16,4.700000e-08,0.166633,0.322648,1
2,c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO...,True,0.755549,0.516673,0.423380,0.415340,0.360944,0.400697,0.428148,0.348007,...,5.770000e-14,2.960000e-11,0.168378,0.16738,1.480000e-18,2.320000e-16,4.700000e-08,0.166633,0.949987,1
3,C1CCN(CC1)Cc1cccc(c1)OCCCNC(=O)C,True,0.519074,0.068097,0.166625,0.260552,0.212714,0.185779,0.328404,0.255385,...,5.770000e-14,2.960000e-11,0.168378,0.16738,1.480000e-18,2.320000e-16,9.991992e-01,0.166633,0.818175,1
4,Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)...,True,0.362651,0.590649,0.607809,0.520746,0.607038,0.538488,0.471718,0.569018,...,5.770000e-14,2.960000e-11,0.168378,0.16738,1.480000e-18,2.320000e-16,4.700000e-08,0.166633,0.689000,1


In [166]:
del data["('RDKit2DNormalized_calculated', <class 'bool'>)"]
del data["smiles"]
#data.insert(0, 'smiles', first_column)
data.head()

,"('BalabanJ', <class 'numpy.float64'>)","('BertzCT', <class 'numpy.float64'>)","('Chi0', <class 'numpy.float64'>)","('Chi0n', <class 'numpy.float64'>)","('Chi0v', <class 'numpy.float64'>)","('Chi1', <class 'numpy.float64'>)","('Chi1n', <class 'numpy.float64'>)","('Chi1v', <class 'numpy.float64'>)","('Chi2n', <class 'numpy.float64'>)","('Chi2v', <class 'numpy.float64'>)",...,"('fr_sulfone', <class 'numpy.float64'>)","('fr_term_acetylene', <class 'numpy.float64'>)","('fr_tetrazole', <class 'numpy.float64'>)","('fr_thiazole', <class 'numpy.float64'>)","('fr_thiocyan', <class 'numpy.float64'>)","('fr_thiophene', <class 'numpy.float64'>)","('fr_unbrch_alkane', <class 'numpy.float64'>)","('fr_urea', <class 'numpy.float64'>)","('qed', <class 'numpy.float64'>)",p_np
0,0.000020,0.111833,0.106451,0.177350,0.187815,0.110054,0.168672,0.119391,0.185866,0.119537,...,5.770000e-14,2.960000e-11,0.168378,0.16738,1.480000e-18,2.320000e-16,4.700000e-08,0.166633,0.930688,1
1,0.921082,0.077697,0.324169,0.379878,0.467164,0.240353,0.344156,0.422452,0.438361,0.462373,...,5.770000e-14,2.960000e-11,0.168378,0.16738,1.480000e-18,2.320000e-16,4.700000e-08,0.166633,0.322648,1
2,0.755549,0.516673,0.423380,0.415340,0.360944,0.400697,0.428148,0.348007,0.528003,0.416469,...,5.770000e-14,2.960000e-11,0.168378,0.16738,1.480000e-18,2.320000e-16,4.700000e-08,0.166633,0.949987,1
3,0.519074,0.068097,0.166625,0.260552,0.212714,0.185779,0.328404,0.255385,0.291954,0.201587,...,5.770000e-14,2.960000e-11,0.168378,0.16738,1.480000e-18,2.320000e-16,9.991992e-01,0.166633,0.818175,1
4,0.362651,0.590649,0.607809,0.520746,0.607038,0.538488,0.471718,0.569018,0.615152,0.792858,...,5.770000e-14,2.960000e-11,0.168378,0.16738,1.480000e-18,2.320000e-16,4.700000e-08,0.166633,0.689000,1


In [167]:

data.shape

(2039, 201)

In [168]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2039 entries, 0 to 2038
Columns: 201 entries, ('BalabanJ', <class 'numpy.float64'>) to p_np
dtypes: float64(199), int64(2)
memory usage: 3.1 MB


In [169]:
data.describe()

,"('BalabanJ', <class 'numpy.float64'>)","('BertzCT', <class 'numpy.float64'>)","('Chi0', <class 'numpy.float64'>)","('Chi0n', <class 'numpy.float64'>)","('Chi0v', <class 'numpy.float64'>)","('Chi1', <class 'numpy.float64'>)","('Chi1n', <class 'numpy.float64'>)","('Chi1v', <class 'numpy.float64'>)","('Chi2n', <class 'numpy.float64'>)","('Chi2v', <class 'numpy.float64'>)",...,"('fr_sulfone', <class 'numpy.float64'>)","('fr_term_acetylene', <class 'numpy.float64'>)","('fr_tetrazole', <class 'numpy.float64'>)","('fr_thiazole', <class 'numpy.float64'>)","('fr_thiocyan', <class 'numpy.float64'>)","('fr_thiophene', <class 'numpy.float64'>)","('fr_unbrch_alkane', <class 'numpy.float64'>)","('fr_urea', <class 'numpy.float64'>)","('qed', <class 'numpy.float64'>)",p_np
count,2039.000000,2039.000000,2039.000000,2.039000e+03,2.039000e+03,2039.000000,2.039000e+03,2039.000000,2039.000000,2.039000e+03,...,2.039000e+03,2.039000e+03,2039.000000,2039.000000,2.039000e+03,2.039000e+03,2.039000e+03,2039.000000,2039.000000,2039.000000
mean,0.604334,0.291380,0.337879,3.573345e-01,3.555512e-01,0.330076,3.722977e-01,0.364649,0.409469,4.051476e-01,...,9.808730e-03,7.846984e-03,0.177759,0.186164,1.480000e-18,1.618440e-02,8.005408e-02,0.201374,0.578456,0.765081
std,0.312428,0.251739,0.293971,2.893572e-01,2.938552e-01,0.282478,2.921984e-01,0.292215,0.313242,3.164573e-01,...,9.857629e-02,8.825661e-02,0.087846,0.123671,5.779207e-34,1.262152e-01,2.681487e-01,0.166608,0.279008,0.424052
min,0.000020,0.000577,0.000002,1.980000e-08,4.530000e-07,0.000001,2.680000e-07,0.000042,0.000001,2.210000e-07,...,5.770000e-14,2.960000e-11,0.168378,0.167380,1.480000e-18,2.320000e-16,4.700000e-08,0.166633,0.003427,0.000000
25%,0.376547,0.082053,0.082985,9.334640e-02,8.693349e-02,0.085748,1.031724e-01,0.093407,0.112201,1.007599e-01,...,5.770000e-14,2.960000e-11,0.168378,0.167380,1.480000e-18,2.320000e-16,4.700000e-08,0.166633,0.370570,1.000000
50%,0.684349,0.230083,0.243373,2.944113e-01,2.866005e-01,0.256325,3.198660e-01,0.309533,0.364833,3.658837e-01,...,5.770000e-14,2.960000e-11,0.168378,0.167380,1.480000e-18,2.320000e-16,4.700000e-08,0.166633,0.631207,1.000000
75%,0.874519,0.438195,0.560798,5.762799e-01,5.794196e-01,0.538452,6.049746e-01,0.595135,0.661656,6.885461e-01,...,5.770000e-14,2.960000e-11,0.168378,0.167380,1.480000e-18,2.320000e-16,4.700000e-08,0.166633,0.812874,1.000000
max,0.999343,1.000000,0.999913,9.998828e-01,9.999034e-01,0.999915,9.998737e-01,0.999931,0.999863,9.998239e-01,...,1.000000e+00,1.000000e+00,1.000000,1.000000,1.480000e-18,1.000000e+00,1.000000e+00,1.000000,0.999021,1.000000


In [170]:
data.isnull().sum().sum()

0

In [171]:

Xdata = data.iloc[:, 0:200]
Ydata = data.iloc[:, 200:201]

In [172]:
Ydata.head

<bound method NDFrame.head of       p_np
0        1
1        1
2        1
3        1
4        1
...    ...
2034     1
2035     1
2036     1
2037     1
2038     1

[2039 rows x 1 columns]>

In [173]:
# from sklearn.model_selection import train_test_split
# train_data, test_data, train_labels, test_labels = train_test_split(Xdata, Ydata, test_size=0.3, random_state=1)
# print(train_data.shape, train_labels.shape, test_data.shape, test_labels.shape)

from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(Xdata, Ydata, stratify = Ydata , test_size = 0.25)

In [174]:
train_data = np.array(train_data)
test_data = np.array(test_data)

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [175]:
train_data.shape

(1529, 200)

In [176]:
from sklearn.decomposition import PCA as sklearnPCA
sklearn_pca = sklearnPCA(n_components=15)
train_pca = sklearn_pca.fit_transform(train_data)
print(train_pca.shape)

(1529, 15)


In [177]:
# from sklearn import preprocessing

# scaler_data = preprocessing.MinMaxScaler()
# train_data = scaler_data.fit_transform(train_data)
# test_data = scaler_data.transform(test_data)

# scaler_labels = preprocessing.MinMaxScaler()
# train_labels_before = train_labels.values.reshape(-1, 1)
# train_labels = scaler_labels.fit_transform(train_labels.values.reshape(-1, 1))
# test_labels = scaler_labels.transform(test_labels.values.reshape(-1, 1))

# print(train_data.shape, train_labels.shape, test_data.shape, test_labels.shape)
# print("Train labels before scaling: {} {} {}Train labels after scaling: {} {}".format('\n',train_labels_before,'\n', '\n', train_labels))

In [178]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [179]:
# transform to torch tensor

tensor_x = torch.tensor(train_data, dtype=torch.float).to(device) 
tensor_x2 = torch.tensor(test_data, dtype=torch.float).to(device) 

tensor_y = torch.tensor(train_labels, dtype=torch.float).to(device)
tensor_y2 = torch.tensor(test_labels, dtype=torch.float).to(device)

In [180]:
# create your dataset

trainset = TensorDataset(tensor_x, tensor_y) 
testset = TensorDataset(tensor_x2,tensor_y2)

In [181]:
%%capture
try:
    import ray
except:
    !pip install -U ray
    import ray

In [182]:
from functools import partial
import numpy as np
import os
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
from torchsummary import summary

from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

In [183]:
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.suggest.optuna import OptunaSearch  
from ray.tune.suggest.dragonfly import DragonflySearch
from ray.tune.suggest.bayesopt import BayesOptSearch
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.schedulers import MedianStoppingRule
from ray.tune.schedulers import PopulationBasedTraining
from ray.tune.suggest.bohb import TuneBOHB
from ray.tune.suggest.basic_variant import BasicVariantGenerator
from ray.tune.suggest import ConcurrencyLimiter

In [184]:
def load_data(data_dir=None):
    return trainset, testset

In [185]:
from sklearn.metrics import roc_auc_score

def compute_score(model, data_loader, device="cpu"):
    model.eval()
    metric = roc_auc_score
    with torch.no_grad():
        prediction_all= torch.empty(0, device=device)
        labels_all= torch.empty(0, device=device)
        for i, (feats, labels) in enumerate(data_loader):
            feats=feats.to(device)
            labels=labels.to(device)
            prediction = model(feats).to(device)
            prediction = torch.sigmoid(prediction).to(device)
            prediction_all = torch.cat((prediction_all, prediction), 0)
            labels_all = torch.cat((labels_all, labels), 0)                
        try:
            t = metric(labels_all.int().cpu(), prediction_all.cpu()).item()
        except ValueError:
            t = 0
    return t

In [186]:
class Net(nn.Module):
    def __init__(self, config):
        super().__init__()
        
        self.config = config
        self.hidden_dim1 = int(self.config.get("hidden_dim1", 100))
        self.hidden_dim2 = int(self.config.get("hidden_dim2", 100))
        self.hidden_dim3 = int(self.config.get("hidden_dim3", 100))

        self.act1 = self.config.get("act1", "relu") 
        self.act2 = self.config.get("act2", "relu") 
        self.act3 = self.config.get("act3", "relu")

        self.linear1 = nn.Linear(200, self.hidden_dim1)
        self.linear2 = nn.Linear(self.hidden_dim1, self.hidden_dim2)
        self.linear3 = nn.Linear(self.hidden_dim2, self.hidden_dim3)
        self.linear4 = nn.Linear(self.hidden_dim3, 1)
    
    @staticmethod
    def activation_func(act_str):
        if act_str=="tanh":
            return eval("torch."+act_str)
        elif act_str=="selu" or act_str=="relu":   
            return eval("torch.nn.functional."+act_str)

    def forward(self, x):
        output = self.linear1(x)
        output = self.activation_func(self.act1)(output)
        output = self.linear2(output)
        output = self.activation_func(self.act2)(output)
        output = self.linear3(output)
        output = self.activation_func(self.act3)(output)
        output = self.linear4(output)        
        return output

In [187]:
model = Net({})

In [188]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

In [189]:
summary(model, (1,tensor_x.shape[1]))

count_parameters(model)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 100]          20,100
            Linear-2               [-1, 1, 100]          10,100
            Linear-3               [-1, 1, 100]          10,100
            Linear-4                 [-1, 1, 1]             101
Total params: 40,401
Trainable params: 40,401
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.15
Estimated Total Size (MB): 0.16
----------------------------------------------------------------
+----------------+------------+
|    Modules     | Parameters |
+----------------+------------+
| linear1.weight |   20000    |
|  linear1.bias  |    100     |
| linear2.weight |   10000    |
|  linear2.bias  |    100     |
| linear3.weight |   10000    |
|  linear3.bias  |    100     |
| linear4.we

40401

In [190]:
def trainable_func(config, checkpoint_dir=None, data_dir=None, epochs=10):

    net = Net(config)

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    '''
    Define a loss function
    '''
    ## Classification
    criterion = nn.CrossEntropyLoss()

    # Define an optimizer 
    optimizer = optim.Adam(net.parameters(), lr=config.get("lr",0.0003))

    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    # Load data
    trainset, testset = load_data(data_dir)

    # Split the dataset into training and validation sets
    train_size = int(len(trainset) * 0.66)
    train_subset, val_subset = random_split(trainset, [train_size, len(trainset) - train_size])

    # Define data loaders (which combines a dataset and a sampler, and provides an iterable over the given dataset)
    trainloader = torch.utils.data.DataLoader(
        train_subset,
        batch_size=int(config.get("batch_size",32)),
        shuffle=True,
        num_workers=2)
    valloader = torch.utils.data.DataLoader(
        val_subset,
        batch_size=int(config.get("batch_size",32)),
        shuffle=True,
        num_workers=2)

    for epoch in range(epochs):  # loop over the dataset multiple times
        epoch_train_loss = 0.0
        # epoch_steps = 0
        net.train() # Prepare model for training
        for i, data in enumerate(trainloader):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            '''
            Compute train loss without scaling to print
            ''' 
            # outputs = torch.tensor(scaler_labels.inverse_transform(outputs.detach().cpu())).to(device)    
            # labels = torch.tensor(scaler_labels.inverse_transform(labels.cpu())).to(device)  
            # loss_train = criterion(outputs, labels) 
            # epoch_train_loss += loss_train.detach().item()
        # print("[%d] loss: %.3f" % (epoch + 1, epoch_train_loss / len(train_subset)))

        # Validation loss
        # val_loss = 0.0
        # net.eval() # Prepare model for evaluation
        # for i, data in enumerate(valloader):
        #     with torch.no_grad():
        #         inputs, labels = data
        #         inputs, labels = inputs.to(device), labels.to(device)

        #         outputs = net(inputs)

        # Validation score
        score = compute_score(net, valloader, device="cpu")

        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((net.state_dict(), optimizer.state_dict()), path)
        tune.report(score=score)

                # Inverse transform of the labels' scaler  این دو خط رو کامنت کردی!!!!!
                #outputs = torch.tensor(scaler_labels.inverse_transform(outputs.detach().cpu())).to(device)    
                #labels = torch.tensor(scaler_labels.inverse_transform(labels.cpu())).to(device) 

                # loss = criterion(outputs, labels)
                # val_loss += loss.cpu().numpy()

    #     with tune.checkpoint_dir(epoch) as checkpoint_dir:
    #         path = os.path.join(checkpoint_dir, "checkpoint")
    #         torch.save((net.state_dict(), optimizer.state_dict()), path)

    #     tune.report(epoch = epoch, loss=(val_loss / len(val_subset)))
    print("Finished Training")

In [191]:
# def test_score(config, net, device="cpu"):
#     trainset, testset = load_data()

#     testloader = torch.utils.data.DataLoader(
#         testset, batch_size=int(config.get("batch_size",32)), shuffle=False, num_workers=2)
    
#     criterion = nn.CrossEntropyLoss()

#     # Test loss
#     # test_loss = 0.0
#     # net.eval() # Prepare model for evaluation
#     # for i, data in enumerate(testloader):
#     #     with torch.no_grad():
#     #         inputs, labels = data
#     #         inputs, labels = inputs.to(device), labels.to(device)

#     #         outputs = net(inputs)

#     test_score = compute_score(best_trained_model, testloader, device)
#     print("Best trial test set score: {}".format(test_score))

#             # Inverse transform of the labels' scaler
#             # outputs = torch.tensor(scaler_labels.inverse_transform(outputs.detach().cpu())).to(device)    
#             # labels = torch.tensor(scaler_labels.inverse_transform(labels.cpu())).to(device) 

#     #         loss = criterion(outputs, labels)
#     #         test_loss += loss.cpu().numpy()

#     # return test_loss / len(testset)

def test_score(config, net, device="cpu"):
    trainset, testset = load_data()
    best_trial=config
    testloader = torch.utils.data.DataLoader(
            testset, 
            batch_size=int(best_trial["batch_size"]), 
            shuffle=False, 
            num_workers=2)
    test_score = compute_score(net, testloader, device)
    print("Best trial test set score: {}".format(test_score))

In [192]:
ray.shutdown()
ray.init() # Here we use ray.init() to evaluate available_resources for Ray
print(ray.available_resources())
  # Restart Ray defensively in case the ray connection is lost. 

# Start Ray runtime with specific resources (not nessesarily all resources)
# You can change this values based on your machine resources)
#ray.init(num_cpus=2, num_gpus=0) 

"""Check Ray Tune is working properly (for trainable class)"""
# from ray.tune.utils import validate_save_restore
# validate_save_restore(Trainable)
# validate_save_restore(Trainable, use_object_store=True)
# print("Success!")

{'node:172.28.0.2': 1.0, 'object_store_memory': 3966001152.0, 'CPU': 2.0, 'memory': 7932002304.0}


'Check Ray Tune is working properly (for trainable class)'

In [193]:
%%capture
try:
    import optuna
except:
    %pip install optuna
    import optuna

In [194]:
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=2):

    # define data directory here if you want to load data from files
    data_dir = os.path.abspath("./bbbp")
    load_data(data_dir)

    # define the search space of hyperparameters
    config = {
        "act1 ": tune.choice(["relu","tanh","selu"]),
        "act2" : tune.choice(["relu","tanh","selu"]),
        "act3" : tune.choice(["relu","tanh","selu"]),
        "lr": tune.quniform(0.0005, 0.001, 0.0001),
        "batch_size": tune.choice([8, 16, 32]),
        "hidden_dim1" : tune.quniform(50, 200, 10),
        "hidden_dim2" : tune.quniform(50, 200, 10),
        "hidden_dim3" : tune.quniform(50, 200, 10),
    }

    # Optuna search algorithm
    from ray.tune.suggest.optuna import OptunaSearch 
    from ray.tune.suggest import ConcurrencyLimiter
    search_alg = OptunaSearch(
        metric="score", #or accuracy, etc.
        mode="max", #or max
        #seed = 42,
        # points_to_evaluate=[
        # {'lr': 0.0005, 'hidden_size': 150.0, 'readout1_out': 200.0, 'readout2_out': 180.0}
        # ],
        )
    

    # ## Hyperopt
    # search_alg = HyperOptSearch(
    #     # space=config,
    #     metric="score", #or accuracy, etc.
    #     mode="max", #or max
    #     n_initial_points=60,
    #     # gamma = 0.1,
    #     # random_state_seed=42,
    #     # points_to_evaluate=[{"max_norm_val": 2.5, 'GNN_Layers': 3, 'dropout': 0.25, 'dropout1': 0.35, 'dropout2': 0.15, 'lr': 0.001, 'hidden_size': 80, 'readout1_out': 150, 'readout2_out': 100, 'batch_size': 64}],
    #     )

    

    search_alg = ConcurrencyLimiter(search_alg, max_concurrent=10)

    scheduler = ASHAScheduler(
        metric ="score",
        mode="max",
        max_t=max_num_epochs,
        reduction_factor=2, 
        grace_period=4,
        brackets=5
        )
    
    # ## ASHA
    # scheduler = AsyncHyperBandScheduler(
    #     time_attr="training_iteration",
    #     #max_t=training_iteration,
    #     metric="score", #or accuracy, etc.
    #     mode="max", #or max
    #     reduction_factor=2, 
    #     grace_period=4,
    #     brackets=5,
    #     )

    # ## BOHB
    # scheduler = HyperBandForBOHB(
    #     time_attr="training_iteration",
    #     #max_t=training_iteration, 
    #     reduction_factor=8, 
    #     stop_last_trials=True,
    #     metric="score", #or accuracy, etc.
    #     mode="max", #or max
    #     )

    # ## Median
    # scheduler = MedianStoppingRule(
    #     time_attr="training_iteration",
    #     grace_period=10,
    #     min_samples_required=10,
    #     hard_stop = True,
    #     metric="score", #or accuracy, etc.
    #     mode="max", #or max
    #     )


    # ## PBT
    # scheduler = PopulationBasedTraining(
    #     time_attr="training_iteration",
    #     metric="score", #or accuracy, etc.
    #     mode="max", #or max
    #     perturbation_interval=10,  # every 10 `time_attr` units
    #                             # (training_iterations in this case)
    #     # hyperparam_mutations={
    #     #     "lr": [8e-3, 7e-3, 6e-3, 5e-3, 4e-3],
    #     #     "dropout": tune.quniform(0.0, 0.4, 0.05),
    #     #     "dropout1": tune.quniform(0.0, 0.4, 0.05),
    #     #     "dropout2": tune.quniform(0.0, 0.4, 0.05),
    #     #     "max_norm_val":tune.choice([2.5, 3, 3.5, 4]),
    #     #     }
    #     )

    reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["score", "training_iteration"]
        )
    
    # wrap data loading and training for tuning using `partial` 
    # (note that there exist other methods for this purpose)
    result = tune.run(
        partial(trainable_func, data_dir=data_dir, epochs=max_num_epochs),
        scheduler=scheduler,
        search_alg=search_alg,
        num_samples=num_samples,
        config=config,
        verbose=2,
        checkpoint_score_attr="score",
        checkpoint_freq=0,
        keep_checkpoints_num=1,
        # checkpoint_at_end=True,
        # reuse_actors=reuse_actors_status,
        progress_reporter=reporter,
        resources_per_trial={"cpu": 0.5, "gpu": gpus_per_trial},
        stop={"training_iteration": max_num_epochs},                
        )

    best_trial = result.get_best_trial("score", "max", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation score: {}".format(
        best_trial.last_result["score"]))

    best_trained_model = Net(best_trial.config)
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)

    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)

    test_score_value = test_score(best_trial.config, best_trained_model, device)
    # print("Best trial test set score: {}".format(test_score_value))


if __name__ == "__main__":
    # You can change the number of GPUs per trial here:
    main(num_samples=50, max_num_epochs=10, gpus_per_trial=0)

[I 2022-07-26 21:43:13,671] A new study created in memory with name: optuna
2022-07-26 21:43:13,676	WARNING callback.py:106 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


== Status ==
Current time: 2022-07-26 21:43:13 (running for 00:00:00.24)
Memory usage on this node: 1.9/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None
Bracket: Iter 8.000: None
Bracket: 
Bracket: 
Bracket: 
Resources requested: 0.5/2 CPUs, 0/0 GPUs, 0.0/7.39 GiB heap, 0.0/3.69 GiB objects
Result logdir: /root/ray_results/trainable_func_2022-07-26_21-43-13
Number of trials: 1/50 (1 RUNNING)


== Status ==
Current time: 2022-07-26 21:43:19 (running for 00:00:05.44)
Memory usage on this node: 2.2/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None
Bracket: Iter 8.000: None
Bracket: 
Bracket: 
Bracket: 
Resources requested: 2.0/2 CPUs, 0/0 GPUs, 0.0/7.39 GiB heap, 0.0/3.69 GiB objects
Result logdir: /root/ray_results/trainable_func_2022-07-26_21-43-13
Number of trials: 5/50 (1 PENDING, 4 RUNNING)


== Status ==
Current time: 2022-07-26 21:43:24 (running for 00:00:10.46)
Memory usage on this node: 2.5/12.7 GiB


2022-07-26 21:47:17,717	INFO tune.py:748 -- Total run time: 244.09 seconds (243.79 seconds for the tuning loop).


Best trial config: {'act1 ': 'selu', 'act2': 'selu', 'act3': 'selu', 'lr': 0.0008, 'batch_size': 32, 'hidden_dim1': 150.0, 'hidden_dim2': 120.0, 'hidden_dim3': 130.0}
Best trial final validation score: 0.6991720899579867
Best trial test set score: 0.6025961538461538


In [196]:
import random
random.seed(1)
Best_trial_config = {'act1 ': 'tanh', 'act2': 'relu', 'act3': 'tanh', 'lr': 0.0006000000000000001, 'batch_size': 8, 'hidden_dim1': 90.0, 'hidden_dim2': 100.0, 'hidden_dim3': 80.0}
epochs = 10
config = {'act1 ': 'tanh', 'act2': 'relu', 'act3': 'tanh', 'lr': 0.0006000000000000001, 'batch_size': 8, 'hidden_dim1': 90.0, 'hidden_dim2': 100.0, 'hidden_dim3': 80.0}
net = Net(Best_trial_config)
criterion = nn.CrossEntropyLoss()

    # # Define an optimizer 
optimizer = optim.Adam(net.parameters(), lr=config.get("lr",0.0003))

trainset, testset = load_data()

    # Split the dataset into training and validation sets
train_size = int(len(trainset) * 0.66)
train_subset, val_subset = random_split(trainset, [train_size, len(trainset) - train_size])

    # Define data loaders (which combines a dataset and a sampler, and provides an iterable over the given dataset)
trainloader = torch.utils.data.DataLoader(
    train_subset,
    batch_size=int(config.get("batch_size",32)),
    shuffle=True,
    num_workers=2)
valloader = torch.utils.data.DataLoader(
    val_subset,
    batch_size=int(config.get("batch_size",32)),
    shuffle=True,
    num_workers=2)

for epoch in range(epochs):  # loop over the dataset multiple times
     epoch_train_loss = 0.0
        # epoch_steps = 0
     net.train() # Prepare model for training
     for i, data in enumerate(trainloader):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            '''
            Compute train loss without scaling to print
            ''' 
            # outputs = torch.tensor(scaler_labels.inverse_transform(outputs.detach().cpu())).to(device)    
            # labels = torch.tensor(scaler_labels.inverse_transform(labels.cpu())).to(device)  
            # loss_train = criterion(outputs, labels) 
            # epoch_train_loss += loss_train.detach().item()
        # print("[%d] loss: %.3f" % (epoch + 1, epoch_train_loss / len(train_subset)))

        # Validation loss
     val_loss = 0.0
     net.eval() # Prepare model for evaluation
     for i, data in enumerate(valloader):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)

                score = compute_score(net, valloader, device="cpu")

                # # Inverse transform of the labels' scaler
                # outputs = torch.tensor(scaler_labels.inverse_transform(outputs.detach().cpu())).to(device)    
                # labels = torch.tensor(scaler_labels.inverse_transform(labels.cpu())).to(device) 

                # loss = criterion(outputs, labels)
                # val_loss += loss.cpu().numpy()

        # with tune.checkpoint_dir(epoch) as checkpoint_dir:
        #     path = os.path.join(checkpoint_dir, "checkpoint")
        #     torch.save((net.state_dict(), optimizer.state_dict()), path)

        # tune.report(epoch = epoch, loss=(val_loss / len(val_subset)))
print("Finished Training")
device = "cpu"
# best_trained_model.to(device)
test_score_value = test_score(Best_trial_config, net, device)
#print(test_score_value)

Finished Training
Best trial test set score: 0.5055662393162393
